# build_seed_runner による FNmodel 推定デモ
FNmodel\_nh\_100.ipynb の設定をそのまま再利用し、`build_seed_runner` の挙動を確認する実験ノートブックです。以下では環境初期化・設定抽出・推定実行・結果保存の流れで進めます。

In [1]:
import sys
import time
from dataclasses import dataclass
from pathlib import Path
from typing import Any

import matplotlib.pyplot as plt
import pandas as pd
import jax
import jax.numpy as jnp

# リポジトリルートを解決して sys.path に追加}
REPO_ROOT = Path.cwd().resolve()
if REPO_ROOT.name == "notebooks":
    REPO_ROOT = REPO_ROOT.parent
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

# JAX 設定と乱数キー初期化
jax.config.update("jax_enable_x64", True)
base_prng_key = jax.random.PRNGKey(42)

print(f"Project root detected: {REPO_ROOT}")

Project root detected: /Users/yanoshouta/dev/degenerate-diffusion


## 1. 環境および依存関係の初期化
上のセルで Python 標準ライブラリと必要な外部ライブラリをインポートし、プロジェクトルートのパス解決と JAX の乱数キー初期化を済ませました。

In [2]:
from typing import Dict, Tuple

from sympy import Array, symbols

from degenerate_diffusion.evaluation.likelihood_evaluator import LikelihoodEvaluator
from degenerate_diffusion.processes.degenerate_diffusion_process import DegenerateDiffusionProcess

@dataclass
class NotebookSettings:
    model: Any
    evaluator: Any
    true_theta: Tuple[jnp.ndarray, jnp.ndarray, jnp.ndarray]
    bounds_theta1: list[tuple[float, float]]
    bounds_theta2: list[tuple[float, float]]
    bounds_theta3: list[tuple[float, float]]
    t_max: float
    h: float
    burn_out: float
    dt: float
    loop_plan: Dict[int, tuple[str, str, str]]
    initial_theta_stage0: Tuple[jnp.ndarray, jnp.ndarray, jnp.ndarray]

# --- FNmodel 定義（FNmodel_nh_100.ipynb と同一設定）
x_sym, y_sym = symbols("x, y")
theta_10, theta_20, theta_21 = symbols("theta_10 theta_20 theta_21")
theta_30, theta_31 = symbols("theta_30 theta_31")

x = Array([x_sym])
y = Array([y_sym])
theta_1 = Array([theta_10])
theta_2 = Array([theta_20, theta_21])
theta_3 = Array([theta_30, theta_31])

A = Array([theta_20 * y_sym - x_sym + theta_21])
B = Array([[theta_10]])
H = Array([(y_sym - y_sym**3 - x_sym + theta_31) / theta_30])

FNmodel = DegenerateDiffusionProcess(
    x=x,
    y=y,
    theta_1=theta_1,
    theta_2=theta_2,
    theta_3=theta_3,
    A=A,
    B=B,
    H=H,
 )
FN_likelihood = LikelihoodEvaluator(FNmodel)

true_theta1 = jnp.array([0.3])
true_theta2 = jnp.array([1.5, 0.8])
true_theta3 = jnp.array([0.1, 0.0])
true_theta = (true_theta1, true_theta2, true_theta3)

t_max = 100.0
burn_out = 50.0
h = 0.05
dt = 0.001

bounds_theta1 = [(0.1, 0.5)]
bounds_theta2 = [(0.5, 2.5), (0.5, 1.5)]
bounds_theta3 = [(0.01, 0.3), (-1.0, 1.0)]

loop_plan: Dict[int, tuple[str, str, str]] = {
    1: ("B", "B", "B"),
    2: ("M", "M", "M"),
    3: ("M", "M", "M"),
    4: ("M", "M", "M"),
    5: ("M", "M", "M"),
    6: ("M", "M", "M"),
 }

initial_theta_stage0 = (
    jnp.array([0.2]),
    jnp.array([0.5, 0.5]),
    jnp.array([0.2, 0.1]),
 )

settings = NotebookSettings(
    model=FNmodel,
    evaluator=FN_likelihood,
    true_theta=true_theta,
    bounds_theta1=bounds_theta1,
    bounds_theta2=bounds_theta2,
    bounds_theta3=bounds_theta3,
    t_max=t_max,
    h=h,
    burn_out=burn_out,
    dt=dt,
    loop_plan=loop_plan,
    initial_theta_stage0=initial_theta_stage0,
 )

print(settings)

NotebookSettings(model=DegenerateDiffusionProcess(x=[x], y=[y], theta_1=[theta_10], theta_2=[theta_20, theta_21], theta_3=[theta_30, theta_31], A=SymbolicArtifact(expr=[theta_20*y + theta_21 - x], func=<function _lambdifygenerated at 0x147a745e0>), B=SymbolicArtifact(expr=[[theta_10]], func=<function _lambdifygenerated at 0x147a74fe0>), H=SymbolicArtifact(expr=[(theta_31 - x - y**3 + y)/theta_30], func=<function _lambdifygenerated at 0x147a75e40>)), evaluator=<degenerate_diffusion.evaluation.likelihood_evaluator.LikelihoodEvaluator object at 0x1470b4a10>, true_theta=(Array([0.3], dtype=float64), Array([1.5, 0.8], dtype=float64), Array([0.1, 0. ], dtype=float64)), bounds_theta1=[(0.1, 0.5)], bounds_theta2=[(0.5, 2.5), (0.5, 1.5)], bounds_theta3=[(0.01, 0.3), (-1.0, 1.0)], t_max=100.0, h=0.05, burn_out=50.0, dt=0.001, loop_plan={1: ('B', 'B', 'B'), 2: ('M', 'M', 'M'), 3: ('M', 'M', 'M'), 4: ('M', 'M', 'M'), 5: ('M', 'M', 'M'), 6: ('M', 'M', 'M')}, initial_theta_stage0=(Array([0.2], dtype

## 3. build\_seed\_runner による推定実行
抽出した設定を `SeedRunnerConfig` に流し込み、`build_seed_runner` が返す JIT 関数で複数シードを処理します。処理時間も合わせて計測します。

In [3]:
import importlib

import degenerate_diffusion.estimation.parameter_estimator as param_est_new
import degenerate_diffusion.estimation.loop_estimation_algorithm as loop_alg_new

importlib.reload(param_est_new)
loop_alg_new = importlib.reload(loop_alg_new)

SeedRunnerConfig = loop_alg_new.SeedRunnerConfig
build_seed_runner = loop_alg_new.build_seed_runner

seed_runner_config = SeedRunnerConfig(
    true_theta=settings.true_theta,
    t_max=settings.t_max,
    h=settings.h,
    burn_out=settings.burn_out,
    dt=settings.dt,
    bounds_theta1=settings.bounds_theta1,
    bounds_theta2=settings.bounds_theta2,
    bounds_theta3=settings.bounds_theta3,
    newton_kwargs={},
    nuts_kwargs={},
    one_step_kwargs={},
)

seed_runner = build_seed_runner(
    evaluator=settings.evaluator,
    model=settings.model,
    plan=settings.loop_plan,
    config=seed_runner_config,
)

# seeds をまとめて処理するために vmap でベクトル化
vectorized_seed_runner = jax.jit(jax.vmap(seed_runner, in_axes=(0, None)))

seeds = jnp.arange(400, dtype=jnp.int32)
start = time.perf_counter()
theta1_stage0_batch, theta1_final_batch, theta2_stage0_batch, theta3_final_batch = vectorized_seed_runner(
    seeds,
    settings.initial_theta_stage0,
)
elapsed = time.perf_counter() - start

(theta1_stage0_batch, theta1_final_batch, theta2_stage0_batch, theta3_final_batch) = jax.device_get(
    (theta1_stage0_batch, theta1_final_batch, theta2_stage0_batch, theta3_final_batch)
)
# theta1_stage0_batch, theta1_final_batch, theta2_stage0_batch, theta3_final_batchを csvファイルとして保存
df_results = pd.DataFrame({
    "seed": seeds,
    "theta1_stage0": theta1_stage0_batch.flatten(),
    "theta1_final": theta1_final_batch.flatten(),
    "theta2_stage0_0": theta2_stage0_batch[:, 0],
    "theta2_stage0_1": theta2_stage0_batch[:, 1],
    "theta3_final_0": theta3_final_batch[:, 0],
    "theta3_final_1": theta3_final_batch[:, 1],
})
df_results.to_csv("fn_model_seed_runner_results.csv", index=False)

ValueError: Per-column arrays must each be 1-dimensional